In [1]:
import numpy as np
import pandas as pd

In [2]:
import content_rocchio
import collaborative_sgd

In [3]:
ratings = pd.read_json('./ratings.jsonl', lines=True)
content = pd.read_json('./content.jsonl', lines=True)
targets = pd.read_csv('targets.csv', encoding='latin-1', sep=',')

In [4]:
_content = content[['ItemId', 'Actors', 'Director', 'Genre', 'Title', 'imdbVotes', 'imdbRating', 'BoxOffice', 'Metascore']]
_content['Actors'] = _content.Actors.astype(str)
# df_1['plot_keywords'] = df_1.plot_keywords.astype(str) # get keywords from the 'Plot' field
_content['Genre'] = _content.Genre.astype(str)
_content['Director'] = _content.Director.astype(str)


# Removing spaces between names
_content['Director'] = _content['Director'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(', ')))
_content['Actors'] = _content['Actors'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(', ')))

# Discarding the pipes between the plot keywords' and getting only the first five words
# df_1['plot_keywords'] = df_1['plot_keywords'].map(lambda x: x.split('|')) # 'Plot' field but only the keywords
# df_1['plot_keywords'] = df_1['plot_keywords'].apply(lambda x: " ".join(x)) #  'Plot' field but only the keywords

# Discarding the pipes between the genres 
# _content['Genre'] = _content['Genre'].map(lambda x: x.lower().replace(', ', ' '))
_content['Genre'] = _content['Genre'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(',')))
# _content['Features'] = _content['Actors'] +' '+ _content['Director'] +' '+ _content['Genre']
# _content['Genre'] = _content['Genre'].str.replace(r'[\']', '', regex=True)
# _content['Genre'] = _content['Genre'].str.replace(' ', '', regex=True)
_content['Features'] = _content['Genre']

_content['imdbVotes'] = _content['imdbVotes'].str.replace('N/A', '1', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].str.replace(',', '', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].astype(str).astype('float64')
_content['imdbVotes'] = _content['imdbVotes'].fillna(0)

_content['imdbRating'] = _content['imdbRating'].str.replace('N/A', '1.0', regex=True)
_content['imdbRating'] = _content['imdbRating'].astype(str).astype('float64')
_content['imdbRating'] = _content['imdbRating'].fillna(0)

_content['BoxOffice'] = _content['BoxOffice'].str.replace('N/A', '1.0', regex=True)
_content['BoxOffice'] = _content['BoxOffice'].str.replace(',', '', regex=True)
_content['BoxOffice'] = _content['BoxOffice'].str.replace('$', '', regex=True)
_content['BoxOffice'] = _content['BoxOffice'].astype(str).astype('float64')
_content['BoxOffice'] = _content['BoxOffice'].fillna(0)

_content['Metascore'] = _content['Metascore'].str.replace('N/A', '1.0', regex=True)
_content['Metascore'] = _content['Metascore'].astype(str).astype('float64')
_content['Metascore'] = _content['Metascore'].fillna(0)

/tmp/ipykernel_23204/2663391625.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Actors'] = _content.Actors.astype(str)
/tmp/ipykernel_23204/2663391625.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Genre'] = _content.Genre.astype(str)
/tmp/ipykernel_23204/2663391625.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [5]:
# sgd = collaborative_sgd.Sgd(epochs=39, learning_ratio=0.0005, n_factors=4, regulation=0.09, max_rate=10.)
# # sgd = collaborative_sgd.Sgd(epochs=4, learning_ratio=0.0005, n_factors=1, regulation=0.09)
# for row in ratings.itertuples():
#     sgd.add_rating(row.UserId, row.ItemId, row.Rating)
# sgd.fit()


In [6]:
# sgd_predictions = targets[['UserId','ItemId']].copy()
# print(sgd_predictions.shape)
# print(ratings.shape)
# sgd_predictions['PredictedRating'] = sgd_predictions.apply(lambda row: sgd.predict(row['UserId'], row['ItemId']), axis=1)



In [7]:
rocchio = content_rocchio.Rocchio(ratings)
rocchio.fit(_content[['ItemId', 'Features']])


In [8]:
rocchio_similarities = rocchio.predict(targets)

/home/vrjuliao/workfolder/BCC/sistemas-de-recomendacao/tp2/content_rocchio.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consine_sim['user_DOT_item'] = 0
/home/vrjuliao/workfolder/BCC/sistemas-de-recomendacao/tp2/content_rocchio.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consine_sim['user_distance'] = 0
/home/vrjuliao/workfolder/BCC/sistemas-de-recomendacao/tp2/content_rocchio.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [12]:
# recommendations = pd.merge(sgd_predictions, rocchio_similarities, on=['UserId','ItemId'])
recommendations = rocchio_similarities
recommendations = recommendations.merge(_content[['ItemId', 'imdbVotes']], on='ItemId')
# recommendations = recommendations.merge(_content[['ItemId', 'imdbVotes', 'imdbRating', 'BoxOffice', 'Metascore']], on='ItemId')
# recommendations['SimilarityPredicted'] = recommendations['Similarity']*recommendations['PredictedRating']
recommendations['SimilarityIMDB'] = recommendations['Similarity']*recommendations['imdbVotes']
# recommendations = recommendations.sort_values(['UserId', 'SimilarityIMDB', 'imdbVotes', 'BoxOffice', 'Metascore', 'PredictedRating', 'imdbRating' ], ascending=[True, False, False, False, False, False, False])
recommendations = recommendations.sort_values(['UserId', 'SimilarityIMDB', 'imdbVotes'], ascending=[True, False, False])
recommendations.head()


,UserId,ItemId,Similarity,imdbVotes,SimilarityIMDB
2546,0006246bee,ce297c0305,0.739390,411358.0,304153.843249
1741,0006246bee,9277d31cc9,0.788682,352366.0,277904.820956
2434,0006246bee,c1ee6829f5,0.603835,430209.0,259775.196494
2237,0006246bee,b273133e05,0.431311,557708.0,240545.384895
204,0006246bee,1e5bdbcb76,0.431311,475339.0,205018.760194


In [ ]:
# recommendations = pd.merge(sgd_predictions, rocchio_similarities, on=['UserId','ItemId'])
# recommendations = recommendations.merge(_content[['ItemId', 'imdbVotes', 'imdbRating', 'BoxOffice']], on='ItemId')
# recommendations['SimilarityPredicted'] = recommendations['Similarity']*recommendations['PredictedRating']
# recommendations['SimilarityBoxOffice'] = recommendations['Similarity']*recommendations['BoxOffice']
# recommendations['SimilarityIMDB'] = recommendations['Similarity']*recommendations['imdbVotes']
# recommendations = recommendations.sort_values(['UserId', 'SimilarityIMDB','SimilarityBoxOffice', 'imdbVotes', 'SimilarityPredicted', 'BoxOffice', 'imdbRating'], ascending=[True, False, False, False, False, False, False])
# # recommendations = recommendations.sort_values(['UserId','SimilarityIMDB', 'imdbRating'], ascending=[True, False, False])
# # recommendations = recommendations.sort_values(['UserId','SimilarityIMDB','SimilarityPredicted','Similarity','imdbVotes','PredictedRating'], ascending=[True, False, False, False, False, False])
# # recommendations['Similarity'] = recommendations['Similarity']*recommendations['imdbVotes']
# # recommendations = recommendations.sort_values(['UserId','Similarity','imdbVotes'], ascending=[True, False, False])
# recommendations.head()

In [13]:
output_file = open('out-17.csv','w')
print('UserId,ItemId', file=output_file)
pd.set_option("display.max_rows", None, "display.max_columns", None)
for row in recommendations.itertuples():
    print('{},{}'.format(row.UserId, row.ItemId), file=output_file)

In [ ]:
print(rocchio_similarities.shape)
print(recommendations.shape)

(616200, 3)
(616200, 10)


In [ ]:
content.columns

Index(['ItemId', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre',
       'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards',
       'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response', 'totalSeasons',
       'Season', 'Episode', 'seriesID'],
      dtype='object')